# The Optimal Set: utilizing GAMS to optimize the ordering of songs for a DJ set

Max Bublik

CS 524 - introduction to optimization

#### These days, everyone can DJ. The hobby has exploded in popularity as more people realize the fun and relative ease one can have ordering their favorite songs into a consistent stream of music, even utilizing available filters, transition effects or additional music hardware to personalize the set to their interests.

#### But there's just one problem...

### __None of them are optimized__

#### This file utilizes a min-cost Network flow optimization model to determine the optimal ordering of a Spotify playlist with respect to various parameters given by the Spotify API. I have provided the base code to allow the creation of your own CSV file to optimize over any playlist desired, however I will be providing a CSV file named songdata.csv in order to display the process of my model.

I am collecting my data through a utilization of the Spotify API that allows me to gather data on individual track's from a spotify playlist. This allows me to use any playlist on spotify to order the songs, optimizing for different features available through the API

Since the API requires a unique personal Client ID and Client Secret, I elected to save my data in a .csv file that my model will run over. However, code blocks on the bottom of this notebook will instruct you how to add your own spotify playlists to my optimization model

Certain features can be ignored from the response, such as the song ID (which is used to check if the song is the same when organizing data), uri, analysis_url id and track_href.

Before inserting our data into our optimization model, we must remove any possible infiltration of the playlist by media not appropriate for a live DJ set. The two features to account for are Liveness and Speechiness. Any liveness value above 0.8 is most likely to be a live recording of the song, which would not be ideal to perform with. Any speechiness value above 0.66 is likely media that is entirely composed of speech, like a podcast or radio show, which would sound quite weird to play. We also remove any songs that would tamper witht the model, like missing key values represented by values of -1. 

The remaining features to consider are the following: 

>Danceability - value between 0 and 1 that marks how danceable a song is

>energy - value between 0 and 1 that marks how intense a song is, marked by being fast, loud and noisy

>key - integer representation of the musical key the song is in (more info below)

>loudness - the average decibles of a track, ranging from -60 to 0

>mode - indicates if the song is major (1) or minor (0) key

>speechiness - as mentioned, value from 0 to 1 indicating how much speech is present in a track

>acousticness - measure between 0 and 1 indicating how likely it is the track was recorded from acoustic source

>instrumentalness - measure between 0 and 1 indicating how likely a track contains no vocals. 

>valence - measure between 0 and 1 indicating the musical positiveness, values closer to 1 are more happy and upbeat while 0 are sad or depressing

>durration_ms - how many miliseconds long a song is

>time_signature - indicates the time signature as an integer divided by 4, so a value of 4 is in 4/4 time, 3 is in 3/4 time, etc.

let's optimize over these values. 

In [1]:
#first use the import statements below

from dotenv import load_dotenv
import os
import base64
from requests import post, get 
import json
import pandas as pd
import csv
import numpy as np
import io
import chardet


In [2]:
#these are used to assure the ending .lst file can be read in this .ipynb file. Given the size of the .lst file, fair warning it might take a while

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

def convert_mac_to_utf8(input_file, output_file):
    
    # Read encoded file
    with open(input_file, 'r', encoding='mac_roman') as f:
        content = f.read()

    # Write UTF-8-encoded file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)

We start by loading our gams container

In [3]:
%load_ext gams.magic
m = gams.exchange_container

--- Warning: The GAMS version (44.3.0) differs from the API version (45.1.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.1.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.1.0).


In [4]:
#The following uses my songdata.csv file to download a pandas dataframe to optimize over. 

df = pd.read_csv('songdata.csv')
df = df.rename(columns={"Unnamed: 0": "unnamed"})
df = df.set_index('unnamed')

#the stackdf is used for GAMS to understand where to put the data

stackdf = df.stack().reset_index()

i = m.addSet('i', description = 'song label', records = stackdf['level_1'].unique())
j = m.addSet('j', description = 'parameter', records = df.index)

d = m.addParameter('d',[j, i] , records=stackdf)

display(d.pivot())

,want u want u want u,what we had was real,off the ground,blessyou,wow,talk tonight,leavemealone,the sea,shades of love (feat. the joy),i don't want to fall in love,...,find you (feat. denitia),clown from two towns over,too close,like you,omen (feat. lyrah),peace in silence,glide,havanna - bella boo remix,sink,source
danceability,0.605000,0.7550,0.363,0.7090,0.8520,0.7660,0.68000,0.66900,0.65000,0.743000,...,0.6340,0.7210,0.6780,0.470,0.71200,0.575000,0.5500,0.67700,0.0,0.0
energy,0.947000,0.7640,0.768,0.6960,0.7500,0.9650,0.88100,0.84000,0.80100,0.763000,...,0.7930,0.5510,0.8690,0.571,0.75900,0.749000,0.5820,0.67100,0.0,0.0
key,4.000000,10.0000,9.000,4.0000,11.0000,8.0000,0.00000,4.00000,8.00000,1.000000,...,6.0000,11.0000,4.0000,2.000,5.00000,6.000000,3.0000,1.00000,0.0,0.0
loudness,-4.165000,-7.4230,-9.770,-10.8100,-6.2470,-5.9160,-4.50300,-6.46600,-5.62400,-5.207000,...,-5.7230,-13.3070,-7.3650,-7.227,-5.48000,-8.078000,-12.9670,-9.37900,0.0,0.0
mode,1.000000,0.0000,1.000,1.0000,1.0000,1.0000,1.00000,0.00000,0.00000,1.000000,...,0.0000,0.0000,0.0000,0.000,0.00000,1.000000,0.0000,0.00000,0.0,0.0
speechiness,0.125000,0.0967,0.327,0.0632,0.0631,0.0795,0.05530,0.04090,0.07640,0.038700,...,0.1010,0.0472,0.0357,0.114,0.03430,0.174000,0.0423,0.03520,0.0,0.0
acousticness,0.017000,0.1320,0.178,0.2100,0.0251,0.0178,0.00499,0.00677,0.02570,0.061400,...,0.2500,0.0344,0.0966,0.668,0.00235,0.246000,0.3420,0.00183,0.0,0.0
instrumentalness,0.000073,0.0102,0.464,0.5980,0.5560,0.8440,0.02530,0.33700,0.00239,0.000289,...,0.0207,0.8580,0.0876,0.282,0.28600,0.000312,0.9520,0.81000,0.0,0.0
valence,0.549000,0.3630,0.509,0.1920,0.6720,0.6760,0.10500,0.77700,0.60500,0.591000,...,0.3670,0.4660,0.6300,0.208,0.79700,0.357000,0.2340,0.51000,0.0,0.0
tempo,136.991000,127.0410,164.882,136.9820,124.9810,148.0310,174.06200,119.01100,125.00800,134.011000,...,124.9370,130.0160,129.9930,125.117,131.03100,139.783000,127.0050,122.00200,0.0,0.0


For more information on each parameter and corresponding values, check the documentation for track audio featrues from https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features 

When Playing a DJ set, one song only transitions well if it is within a certain range of musical key. my data is ordered by Pitch Class notation, where each integer is associated with a specific musical key 
(so 0 = C, 1 = C#/D♭, 2 = D etc...) and the mode parameter tells me if it is a Major key (value of 1) or Minor key (value of 0). 

The generally accepted range is +- 1 key for songs in the same mode, and equal keys only for songs not in the same mode. This means I can have a transition from 0 -> 1, or 0 -> 11 (since the key's are mod 12) 
if the songs are both Major or Minor, but only 0 -> 0 for a combination of Major and Minor songs. This will define my acceptable arc's between songs, and I will seek to optimize over the other variables with weaker constraints. I also connect every song to my sink and source nodes with direction in mind to initialize my model.

I create individual parameters for each row in my dataframe to make it easier to work with, and create supply to use in network flow

In [5]:
%%gams 
set arcs(i, i);
alias(i, k, l);


parameter danceability(i), energy(i), key(i), loudness(i), mode(i), speechiness(i), acousticness(i), instrumentalness(i), valence(i), tempo(i), durration(i), time_signature(i), supply(i);

danceability(i) = d('danceability', i);
energy(i) = d('energy', i);
key(i) = d('key', i);
loudness(i) = d('loudness', i);
mode(i) = d('mode', i);
speechiness(i) = d('speechiness', i);
acousticness(i) = d('acousticness', i);
instrumentalness(i) = d('instrumentalness', i);
valence(i) = d('valence', i);
tempo(i) = d('tempo', i);
durration(i) = d('duration_ms', i);
time_signature(i) = d('time_signature', i);


scalar timelimit "the set should be a minimum of one hour (3.6 million milliseconds)" /3600000/;

variable cost, arccost(i, i);
positive variable flow(i, i), position(i), overtime;
binary variable flowat(i, i), songUsed(i);


Loop((i, k),
    if (sameas(i, k),
        arcs(i, k) = no;
    else
        if (mode(i) = mode(k), 
            if ((abs(key(i) - key(k)) <= 1) or (abs(key(i) - key(k)) = 11),
                arcs(i, k) = yes;
            else
                arcs(i, k) = no;
            );
        else
            if (key(i) = key(k),
                arcs(i, k) = yes;
            else
                arcs(i, k) = no;
            );
    );
);
);

arcs('source', i) = yes;
arcs(i, 'source') = no;
arcs(i, 'sink') = yes;
arcs('sink', i) = no;
arcs('source', 'sink') = no;

supply('source') = 1;
supply('sink') = -1;



--- Warning: The GAMS version (44.3.0) differs from the API version (45.1.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.1.0).


My initial model simply seeks to minimize the difference between tempo's of each sequential song, a good practice for DJ's to have, as well as keep the total set length as close to an hour as feasible. I set the Reslim to 600 so the model cuts off after 10 minutes and returns the best optimized path at that point. I set this limit after it ran for over 40 minutes and I had better things to be doing with my time, but you are free to adjust the limit to account for your own time as well

In [6]:
%%gams
Option Reslim=600;

equations
    objective                       "Objective function"
    flowBalance(i)                  "Flow balance at each song node"
    timeconst                       "Used to account for time over the hour limit"
    useSongConstraintu              "Used to set upper bound for binary variable if I am using a song"
    useSongConstraintl              "Used to set lower bound for binary variable if I am using a song"
    costarcs                        "The cost equation per arc. I will change this equation to optimize over different parameters"
    oneIncomingFlowConstraint(i)    "Node can have one arc flow in"
    oneOutgoingFlowConstraint(i)    "Node can have one arc flow out"
    noflowtoself(i, i)              "Song's cannot flow into themselves, prevent cycling"
    pathorderconst(i, i)            "Establishes an order of songs"
    ;

objective..
    cost =e= sum(arcs(i, k), arcCost(i, k)) + (overtime/10000);

flowBalance(i)..
    sum(k$arcs(i, k), flowat(i,k)) - sum(l$arcs(l, i), flowat(l,i)) =e= supply(i);
    
timeconst..
    sum(i, songUsed(i)*durration(i)) =e= timelimit + overtime;
    
useSongConstraintu(i)..
    songUsed(i) =l= 2*sum(k$arcs(i, k), flowat(i, k));
    
useSongConstraintl(i)..
    songUsed(i) =g= sum(k$arcs(i, k), flowat(i, k));
    
costarcs(i, k)$(arcs(i, k))..
    arcCost(i, k) =e= abs(tempo(i) - tempo(k))*songUsed(i);

oneIncomingFlowConstraint(i)..
    sum(k$arcs(k, i), flowat(k, i)) =l= 1;

oneOutgoingFlowConstraint(i)..
    sum(k$arcs(i, k), flowat(i, k)) =l= 1;
    
noflowtoself(i, i)..
    flowat(i, i) =e= 0;

pathorderconst(i, k)$(not sameas(i, 'source') and arcs(i, k))..
    position(k) - position(i) =g= 1 - 100*(1 - flowat(i, k));
    

    
model optimalsong /all/;


solve optimalsong using MIP min cost;

parameter eachcost(i, k);
eachcost(i, k) = arcCost.l(i, k)$(flowat.l(i, k))
display flowat.l, songUsed.l, position.l, cost.l, overtime.l;

--- Warning: The GAMS version (44.3.0) differs from the API version (45.1.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.1.0).


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Integer (8),12730.4517,2869,2702,MIP,CPLEX,225.875


In [7]:
#this is to change lst files into utf-8 encoding so display statements can be shown witout manually checking
#Not necessary, and can take a bit of time so be cautious

folder_path = os.getcwd()
files = [str(f.path) for f in os.scandir(folder_path) if ".lst" in str(f.path)]
for f in files:
    if detect_encoding(f) == 'MacRoman':
            convert_mac_to_utf8(f, f)
            print("converted " + f + " to utf8 encoding")
%gams_lst -e

converted c:\Users\maxwe\OneDrive\Documents\personal_projects\cs524_DJ_project\gj_ef7c01ea_base_3.lst to utf8 encoding
E x e c u t i o n


----    128 VARIABLE flowat.L  

                                                                  struggle       caf√®      fading  a break i~  po√´te d√~       blind  missing y~     sambaya      gemini  clown fro~    like you        sink

struggle                                                                                                 1.000
caf√®                                                                                                                                                                                        1.000
fading                                                                                                                                       1.000
a break in the clouds - main mix                                                                                                 1.000
po√´te d√©chu                   

Now I can save the flow variable to extract the order of my songs. I use an assertion that songUsed and flowat is the same length and contains the same variables to check for inconsistencies


In [8]:
#a function made to return my song path as an ordered list that removes the sink and source nodes

def order_songs(x):
    flowin = dict()
    flowincheck = dict()
    for i in range(len(x)):
        if x['level'][i] != 0:
            flowin[x['i_0'][i]] = x['i_1'][i]
    
    ordered = []
    flowinkey = list(flowin.keys())
    
    current = 'source'
    end = 'sink'
    z = True
    while z == True:
        if current == end:
            z = False
        for x in flowinkey:
            if x == current:
                ordered.append(flowin[x])
                current = flowin[x]
    del ordered[-1]
    return ordered

def used_songs(x):
    use = list()
    for i in range(len(x)):
        if x['level'][i] != 0 and x['i'][i] != 'source':
            use.append(x['i'][i])
    return use

In [9]:

y = m.data['flowat'].records
z = m.data['eachcost'].records
used = m.data['songUsed'].records

o = order_songs(y)
u = used_songs(used)

assert len(o) == len(u) # checks the length of each is the same. IF false, model is inconsistent

for x in range(len(o)): #checks that each element of either list is contained within the other
    assert o[x] in u
    assert u[x] in o

o

['clown from two towns over',
 'fading',
 'missing you',
 'cafè',
 'like you',
 'poëte déchu',
 'gemini',
 'struggle',
 'a break in the clouds - main mix',
 'blind',
 'sambaya']

Horray! I now have a list of my songs and can check through the cost of each comparison to see how my model is acting between each song

In [ ]:
z

A simple change I can implement is to account over the percent difference between each bpm instead of the inequality. I can change my costarcs equation to reflect this, having the cost associated be the absolute difference divided by tempo of origional, adding by 1 in the denominator to prevent any divide by 0 erorrs

In [ ]:
%%gams
equations costarcs;

costarcs(i, k)$(arcs(i, k))..
    arcCost(i, k) =e= (abs(tempo(i) - tempo(k)) / (tempo(i) + 1))*songUsed(i);
    
    
model optimalsong /all/;


solve optimalsong min cost use MIP;


In [ ]:

y2 = m.data['flowat'].records
z2= m.data['eachcost'].records
used2 = m.data['songUsed'].records
o2 = order_songs(y2)
u2 = used_songs(used2)

assert len(o2) == len(u2) # checks the length of each is the same. IF false, model is inconsistent

for x in range(len(o2)): #checks that each element of either list is contained within the other
    assert o2[x] in u2
    assert u2[x] in o2

o2

In [ ]:
z2

By adjusting the arcCost equation, I can optimize over different variables and situations. My final model will seek to make the most energetic, upbead and danceable ordering from my playlist using the danceability, energy and valence parameters

In [ ]:
%%gams
equations costarcs;

costarcs(i, k)$(arcs(i, k))..
    arcCost(i, k) =e= (abs(tempo(i) - tempo(k))/(tempo(i) + 1)) * (1 - danceability(i)) * (1 - energy(i)) * (1 - valence(i)) * songUsed(i);
    
    
model optimalsong /all/;


solve optimalsong min cost use MIP;


In [ ]:
y3 = m.data['flowat'].records
z3= m.data['eachcost'].records
used3 = m.data['songUsed'].records
o3 = order_songs(y3)
u3 = used_songs(used3)

assert len(o3) == len(u3) # checks the length of each is the same. IF false, model is inconsistent

for x in range(len(o3)): #checks that each element of either list is contained within the other
    assert o3[x] in u3
    assert u3[x] in o3
o3


With this information on an optimal path through a playlist, I can now optimize over my available parameters to create a set specific to any vibe I want. If I wanted to make a model for a lowkey event (maybe like this https://www.youtube.com/watch?v=kt2mtS7VTG4&ab_channel=FlavourTrip) I would look to optimize over a low energy and higher valence into the arcCost equation. If I wanted to play a mostly instrumental list with occasional blips of vocals, I can use a random variable alongside a sigmoid activation function to switch between optimizing for instrumental or vocal tracks. Through adjusting this equation, the variety possible through my model is only limited to the constraints applied.

Further applications of this model could account for a varied level of optimization over time based on the position of the song. For example, it is common to start a DJ set with lower energy songs to warm the crowd up, then continue to build until you reach a peak. This model could also be used to determine the next best song to play utilizing the ordering of the list in consideration over different parameters. 

In [ ]:
#make sure to cleanup after yourself!
%gams_cleanup --closedown

If you now wish to sovle your own playlist, use the code blocks below and follow the given instructions!

In [ ]:
#the following code will allow you to optimize your own playlist(s) from the spotify API. 

#note that the Spotify API only allows a total of 100 requests at a single time, and limits the number of requests you can make in a given time frame. 
#If you wish to add more songs to your dataframe, you can concatenate multiple pandas dataframes created from multiple playlists
#The API will return the first 100 item's in the playlist, and the concatenation will also account for duplicates.

#You will NEED to enter your Client ID and Client Secret in the .env file. You can find both of these through spotify for developers and by creating a project of your own on their site
#follow this tutorial if you need help https://developer.spotify.com/documentation/web-api/concepts/apps 

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def playlist_IDs(token, playlist_ID):
    
    url = f'https://api.spotify.com/v1/playlists/{playlist_ID}/tracks'
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    
    track_id = dict()
    for x in range(len(json_result['items'])):
        
        #this creates the loop to get the track ID of every song in a playlist
        
        track_id[json_result['items'][x]['track']['name']] = (json_result['items'][x]['track']['id'])
    return(track_id)

def song_data(token, playlist_ID):
    song_dict = playlist_IDs(token, playlist_ID)
    song_data = dict()
    song_names = list(song_dict.keys())
    song_ids = list(song_dict.values())
    song_ids_string = ','.join(song_ids)
    
    #get my song_ids into a continuous comma separated string to use in the request
    #prevents having to make a for loop to each audio-feature and increasing the calls to Spotify API
    
    url = f'https://api.spotify.com/v1/audio-features?ids={song_ids_string}'
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    data = json_result['audio_features']
    #this is the song data!
    
    #test assertions that my lists are the same length, and that the data is from the right track ID
    
    assert len(data) == len(song_names) == len(song_ids), "the lengths of data, song_names and song_ids are not equal"
    for x in range(len(data)):
        assert data[x]['id'] == song_ids[x], f'the song ids at {x} are not equal'
        
        song_data[song_names[x].lower()] = data[x]

       
    
    return song_data

In [ ]:
#you will need to paste the playlist ID below. You can do this by copying the LINK to a PLAYLIST and copying the 22 character ID.
#for example, the playlist ID of my songdata.csv file comes from a playlist called 'Raving Paul'. the link to the playlist is https://open.spotify.com/playlist/7sj6GJx0FD3NPC9RRaUiG3?si=3d54513f3a984a16
#and the playlist ID is '7sj6GJx0FD3NPC9RRaUiG3', located directly after /playlist/ 

playid = 'PASTE YOUR ID HERE'

mytoken = get_token()
csv_file_path = 'mydata.csv'
data = song_data(mytoken, playid)

#for MULTIPLE PLAYLISTS, the following concatenation will remove any duplicates and combine the dictionaries. Simply insert your playlist ID's into the following list and uncomment it

###UNCOMMENT BELOW FOR MULTIPLE PLAYLISTS####
#playids = ['PASTE YOUR IDS INTO THIS LIST']#
#for i in playids:                          #
#    newdata = song_data(mytoken, i)        #
#    data = {**data, **newdata}             #
#############################################

#this is to remove invalid song's from our dictionary of dictionaries

data_val = list(data.values())
data_key = list(data.keys())
for x in range(len(data)):
    data_val_dict = data_val[x]
    liveness = data_val_dict['liveness']
    speechiness = data_val_dict['speechiness']
    key = data_val_dict['key']
    if (liveness >= 0.8) or (speechiness >= 0.66) or (key == -1):
        del data[data_key[x]]
        print('deleted ' + data_key[x] + ' for liveness, speechiness or invalid key')


#now we create a pandas dataframe from our dictionary of dictionaries, removing the parameters we don't need and adding a sink and source node for Network Flow

dataframe = pd.DataFrame(data)
dataframe = dataframe.drop(['analysis_url', 'track_href', 'uri', 'type', 'id', 'liveness'], axis = 0)
dataframe['sink'] = 0
dataframe['source'] = 0
print(dataframe)

In [ ]:

df = dataframe.rename(columns={"Unnamed: 0": "unnamed"})
df = df.set_index('unnamed')

#the stackdf is used for GAMS to understand where to put the data
stackdf = df.stack().reset_index()

i = m.addSet('i', description = 'song label', records = stackdf['level_1'].unique())
j = m.addSet('j', description = 'parameter', records = df.index)

d = m.addParameter('d',[j, i] , records=stackdf)

display(d.pivot())

In [ ]:
%%gams 
set arcs(i, i);
alias(i, k, l);


parameter danceability(i), energy(i), key(i), loudness(i), mode(i), speechiness(i), acousticness(i), instrumentalness(i), valence(i), tempo(i), durration(i), time_signature(i), supply(i);

danceability(i) = d('danceability', i);
energy(i) = d('energy', i);
key(i) = d('key', i);
loudness(i) = d('loudness', i);
mode(i) = d('mode', i);
speechiness(i) = d('speechiness', i);
acousticness(i) = d('acousticness', i);
instrumentalness(i) = d('instrumentalness', i);
valence(i) = d('valence', i);
tempo(i) = d('tempo', i);
durration(i) = d('duration_ms', i);
time_signature(i) = d('time_signature', i);


scalar timelimit "the set should be a minimum of one hour" /3600000/;

variable cost, arccost(i, i);
positive variable flow(i, i), position(i), overtime;
binary variable flowat(i, i), songUsed(i);


Loop((i, k),
    if (sameas(i, k),
        arcs(i, k) = no;
    else
        if (mode(i) = mode(k), 
            if ((abs(key(i) - key(k)) <= 1) or (abs(key(i) - key(k)) = 11),
                arcs(i, k) = yes;
            else
                arcs(i, k) = no;
            );
        else
            if (key(i) = key(k),
                arcs(i, k) = yes;
            else
                arcs(i, k) = no;
            );
    );
);
);

arcs('source', i) = yes;
arcs(i, 'source') = no;
arcs(i, 'sink') = yes;
arcs('sink', i) = no;
arcs('source', 'sink') = no;

supply('source') = 1;
supply('sink') = -1;



If you would like to add any additional constraints, you can do so below by adjusting the costarcs equation

In [ ]:
%%gams
Option Reslim=600;

equations
    objective                       "Objective function"
    flowBalance(i)                  "Flow balance at each song node"
    timeconst                       "Used to account for time over the hour limit"
    useSongConstraintu              "Used to set upper bound for binary variable if I am using a song"
    useSongConstraintl              "Used to set lower bound for binary variable if I am using a song"
    costarcs                        "The cost equation per arc. I will change this equation to optimize over different parameters"
    oneIncomingFlowConstraint(i)    "Node can have one arc flow in"
    oneOutgoingFlowConstraint(i)    "Node can have one arc flow out"
    noflowtoself(i, i)              "Song's cannot flow into themselves, prevent cycling"
    pathorderconst(i, i)            "Establishes an order of songs"
    ;

objective..
    cost =e= sum(arcs(i, k), arcCost(i, k)) + (overtime/10000);

flowBalance(i)..
    sum(k$arcs(i, k), flowat(i,k)) - sum(l$arcs(l, i), flowat(l,i)) =e= supply(i);
    
timeconst..
    sum(i, songUsed(i)*durration(i)) =e= timelimit + overtime;
    
useSongConstraintu(i)..
    songUsed(i) =l= 2*sum(k$arcs(i, k), flowat(i, k));
    
useSongConstraintl(i)..
    songUsed(i) =g= sum(k$arcs(i, k), flowat(i, k));
    
    
###################change costarcs here####################
            
costarcs(i, k)$(arcs(i, k))..
    arcCost(i, k) =e= abs(tempo(i) - tempo(k))*songUsed(i);
             
###########################################################




oneIncomingFlowConstraint(i)..
    sum(k$arcs(k, i), flowat(k, i)) =l= 1;

oneOutgoingFlowConstraint(i)..
    sum(k$arcs(i, k), flowat(i, k)) =l= 1;
    
noflowtoself(i, i)..
    flowat(i, i) =e= 0;

pathorderconst(i, k)$(not sameas(i, 'source') and arcs(i, k))..
    position(k) - position(i) =g= 1 - 100*(1 - flowat(i, k));
    

    
model optimalsong /all/;


solve optimalsong using MIP min cost;

parameter eachcost(i, k);
eachcost(i, k) = arcCost.l(i, k)$(flowat.l(i, k))
display flowat.l, songUsed.l, position.l, cost.l, overtime.l;

In [ ]:
your_flowat = m.data['flowat'].records
your_eachcost = m.data['eachcost'].records
your_order = order_songs(your_flowat)
your_usedsongs = used_songs(m.data['songUsed'].records)

assert len(your_order) == len(your_usedsongs) 

for x in range(len(your_order)): 
    assert your_order[x] in your_usedsongs
    assert your_usedsongs[x] in your_order

### Here is your optimal order!!

In [ ]:
your_order

In [ ]:
your_eachcost

In [ ]:
%gams_cleanup --closedown